# Day 20: Race Condition
First, let's read and parse the input data

In [1]:
with open('aoc20.txt', 'r') as f:
    grid = [list(line.strip()) for line in f.readlines()]

Find start (S) and end (E) positions

In [2]:
def find_positions(grid):
    start = end = None
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            if grid[y][x] == 'S':
                start = (y, x)
            elif grid[y][x] == 'E':
                end = (y, x)
    return start, end

Calculate shortest path time without cheating using BFS

In [3]:
from collections import deque

def normal_path_time(grid, start, end):
    queue = deque([(start, 0)])
    seen = {start}
    
    while queue:
        pos, time = queue.popleft()
        if pos == end:
            return time
        
        y, x = pos
        for ny, nx in [(y+1,x), (y-1,x), (y,x+1), (y,x-1)]:
            if (0 <= ny < len(grid) and 0 <= nx < len(grid[0]) and
                grid[ny][nx] != '#' and (ny,nx) not in seen):
                queue.append(((ny,nx), time+1))
                seen.add((ny,nx))
    return float('inf')

Find all possible cheats and calculate time savings

In [4]:
def find_cheating_paths(grid, base_time):
    start, end = find_positions(grid)
    savings = []
    
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            if grid[y][x] == '#':
                time_to_here = normal_path_time(grid, start, (y-1,x))
                if time_to_here == float('inf'):
                    continue
                    
                # Try jumping through wall within 2 steps
                for jump_y in range(max(0,y-2), min(len(grid),y+3)):
                    for jump_x in range(max(0,x-2), min(len(grid[0]),x+3)):
                        if grid[jump_y][jump_x] == '#':
                            continue
                        time_from_jump = normal_path_time(grid, (jump_y,jump_x), end)
                        if time_from_jump != float('inf'):
                            total_time = (time_to_here + abs(jump_y-y) + 
                                         abs(jump_x-x) + time_from_jump)
                            if total_time < base_time:
                                savings.append(base_time - total_time)
    return savings

Calculate and save result

In [5]:
start, end = find_positions(grid)
base_time = normal_path_time(grid, start, end)
savings = find_cheating_paths(grid, base_time)

result = sum(1 for s in savings if s >= 100)
with open('result.txt', 'w') as f:
    f.write(str(result))